<a href="https://colab.research.google.com/github/AlexandreFenyo/awsgpu/blob/main/notebooks/RAG_llamaindex_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG avec Llamaindex
Je suis le plan d'action fourni par GPT-5-nano : "Je veux créer un notebook dans Google Colab, pour utiliser Llamaindex afin de procéder à de la recherche de type RAG. Je ne veux pas invoquer de LLM ensuite, je me limite à récupérer des chunks via du RAG."

J'installe les biblothèques utiles :

In [ ]:
!pip -q install --upgrade llama-index llama-index-embeddings-huggingface transformers sentence-transformers # faiss-cpu

# Importations Python pour les embeddings

In [ ]:
from google.colab import drive
from pathlib import Path
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.core import Settings

# Monter Drive et pointer vers les documents

In [ ]:
drive.mount('/content/drive', force_remount=True)
docs_dir = "/content/drive/MyDrive/RAG_docs"
docs_dir_path = Path(docs_dir)
print("Répertoire des documents : ", docs_dir_path, "\nMontage réalisé :", docs_dir_path.exists())
docs = SimpleDirectoryReader(input_dir=str(docs_dir_path)).load_data()

Mounted at /content/drive
Répertoire des documents :  /content/drive/MyDrive/RAG_docs 
Montage réalisé : True


# Travailler avec des Embeddings locaux

In [ ]:
embedding_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")
Settings.embed_model = embedding_model

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


# Construire l'index avec VectorStoreIndex.from_documents

Ce que fait VectorStoreIndex.from_documents :
- Convertit tes documents en chunks (selon le TextSplitter que tu utilises), calcule les embeddings et les stocke dans un « vector store » (un index de vecteurs).
- Côté récupération, il interroge ce même vector store pour trouver les chunks les plus pertinents par similarity.
- Tu n’es pas obligé d’avoir une base externe (Weaviate, Pinecone, Qdrant, etc.). Si tu ne précises pas de vector store, llama-index utilise par défaut un stockage vectoriel en mémoire (un FAISS-like store) et peut persister sur disque via StorageContext. (developers.llamaindex.ai)


In [ ]:
index = VectorStoreIndex.from_documents(docs, embed_model=embedding_model)

# Helper pour l'affichage des chunks selon leurs types respectifs

In [ ]:
def extract_text(obj):
  if hasattr(obj, "text"):
    return obj.text
  if hasattr(obj, "node"):
    node = obj.node
  if hasattr(node, "text"):
    return node.text
  if hasattr(node, "get_text"):
    return node.get_text()
  return str(obj)

# Récupération de chunks via une requête

In [ ]:
retriever = index.as_retriever(search_kwargs={"k": 5})
retrieved_nodes = retriever.retrieve("ta requête")

# Affichage des chunks récupérés

In [ ]:
for i, n in enumerate(retrieved_nodes, start=1):
  print(f"--- Chunk {i} ---")
  print(extract_text(n))
  print()

--- Chunk 1 ---
L’IMPORTANCE DE LA SÉCURITÉ
Internet offre des possibilités à la fois riches et nouvelles pour la croissance 
et le développement commercial. Grâce à la diversité des services et des
solutions désormais disponibles sur le réseau, les sociétés sont mieux 
à même de prendre soin de leur clientèle, de générer des synergies entre
des employés géographiquement éloignés et de se créer des opportunités
de nouveaux revenus par l’accès à une base de clients plus large et plus
diversifiée.
Mais si Internet a transformé et nettement amélioré les transactions 
commerciales, ce vaste réseau et les technologies qui lui correspondent 
ont ouvert la porte à un nombre croissant de menaces relatives à la sécurité
contre lesquelles les entreprises doivent se prémunir. Bien que les attaques
des réseaux soient généralement plus graves lorsqu’elles visent des sociétés
qui stockent des données critiques, comme des dossiers confidentiels 
médicaux ou financiers, les conséquences de ces attaque